# GitHubAgent runtime test (mocked)

This notebook performs a runtime test of `doppiozero.agents.gh_deep_search.GitHubAgent` using local, deterministic mocks for the content layer so no network or credentials are required.

Run the cells in order. The notebook does not create any new test files; it only monkeypatches in-memory objects and saves a single `agent_test_output.json` artifact.

In [1]:
# Ensure the repository root is on sys.path so `doppiozero` imports resolve
import sys, os
from pathlib import Path
PROJECT_ROOT = Path(os.getcwd()).resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
print('Inserted PROJECT_ROOT to sys.path:', PROJECT_ROOT)

Inserted PROJECT_ROOT to sys.path: /Users/romanofoti/romanofoti/doppiozero


In [2]:
import types, json
from pprint import pprint
import doppiozero.contents as contents_mod

from doppiozero.agents.gh_deep_search import GitHubAgent

## Monkeypatch local content layer (mocked integration)

This cell replaces `content_manager` and `content_fetcher` helpers with deterministic functions so the agent can run end-to-end without external services.

In [3]:

cm = getattr(contents_mod, 'content_manager', None)
cf = getattr(contents_mod, 'content_fetcher', None)
# If the modules are objects, patch their functions; otherwise create simple namespaces
def fake_search(query, max_results=5):
    # Return deterministic fake search results referencing made-up URLs
    return [
        {'url': f'https://example.com/convo/{i}', 'score': 1.0 - i*0.1} for i in range(max_results)
    ]

def fake_vector_search(query, collection=None, top_k=5):
    # Return search hits with summaries and scores
    return [
        {'url': f'https://example.com/convo/{i}', 'summary': f'Summary of {query} #{i}', 'score': 1.0 - i*0.1} for i in range(top_k)
    ]

def fake_fetch_github_conversation(url, cache_path=None):
    # Return a small conversation dict
    return {'url': url, 'messages': [
        {'author': 'alice', 'text': 'Initial issue description.'},
        {'author': 'bob', 'text': 'Follow-up discussion.'},
    ]}

def fake_summarize(url, prompt_path=None, cache_path=None):
    return f'Compact summary for {url}'

def fake_vector_upsert(text, collection, metadata, model=None):
    # No-op upsert for tests
    return True

# Attach mocks to the content modules (create attributes if missing)
if cm is None:
    contents_mod.content_manager = types.SimpleNamespace()
    cm = contents_mod.content_manager
if cf is None:
    contents_mod.content_fetcher = types.SimpleNamespace()
    cf = contents_mod.content_fetcher

# Patch functions used by the agent
setattr(cm, 'search', fake_search)
setattr(cm, 'vector_search', fake_vector_search)
setattr(cm, 'summarize', fake_summarize)
setattr(cm, 'vector_upsert', fake_vector_upsert)
setattr(contents_mod, 'content_manager', cm)
setattr(cf, 'fetch_github_conversation', fake_fetch_github_conversation)
setattr(contents_mod, 'content_fetcher', cf)

print('Patched content_manager and content_fetcher with deterministic mocks')

Patched content_manager and content_fetcher with deterministic mocks


In [ ]:
# Interactive clarifying answers (edit this cell before running the agent)
from pathlib import Path

clarifying_answers = '''Q: What is the main goal?
A: Find recent authentication failure discussions across repos.

Q: Are there specific repos to focus on?
A: doppiozero and financial-planning.
'''

clarifying_path = Path('clarifying_answers.txt').resolve()
clarifying_path.write_text(clarifying_answers, encoding='utf-8')
print('Wrote clarifying answers to', clarifying_path)
# The agent will read this file if options['clarifying_qa'] points to it.
# If you prefer to answer interactively, edit the clarifying_answers string above and re-run this cell.

## Run the agent (mocked)

Instantiate `GitHubAgent` with lightweight options and run it. The mocked content layer will return deterministic data so the run is fast and offline.

Interactive notes: Edit the clarifying-answers cell (above) if you want to provide clarifying answers manually. If `clarifying_answers.txt` exists when you run the agent cell, it will be used; otherwise the agent will run with no clarifications (or will open an editor if it is configured to do so).

In [ ]:
# Run the agent with mocked content layer

from pathlib import Path
# If the clarifying answers file was written above, prefer it; otherwise leave as None
clarifying_file = Path('clarifying_answers.txt').resolve()
clarifying_qa = str(clarifying_file) if clarifying_file.exists() else None

options = {
    'collection': None,
    'limit': 3,
    'max_depth': 1,
    'editor_file': None,
    # Use clarifying_qa if the file exists; set to None to allow interactive editing
    'clarifying_qa': clarifying_qa,
    'search_modes': ['semantic'],
    'cache_path': None,
    'models': {'fast': 'default', 'reasoning': 'default', 'embed': 'default'},
    'parallel': False,
    'verbose': True,
}
agent = GitHubAgent('What are the recent discussions about authentication failures?', options)
result = agent.run()
print('=== AGENT RUN RESULT ===')
pprint(result)
print('=== SHARED STATE ===')
pprint(agent.shared)

2025-09-03 19:28:51,656 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - === GITHUB CONVERSATIONS RESEARCH AGENT ===
2025-09-03 19:28:51,658 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Request: What are the recent discussions about authentication failures?
2025-09-03 19:28:51,659 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Collection: None
2025-09-03 19:28:51,659 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Max results per search: 3
2025-09-03 19:28:51,661 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Max deep research iterations: 1
2025-09-03 19:28:51,661 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Fast model: default
2025-09-03 19:28:51,662 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Reasoning model: default
2025-09-

7[>4;2m=[>4;m[>4;2m"/var/folders/mp/vh6r5x693yb06mk7jfhk6nkc0000gn/T/tmpnde3ao6v.md" 6L, 130BPlease review the following questions and provide inline answers:

What is the main goal?

Are there specific repos to focus on?

~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                   

KeyboardInterrupt: 

In [ ]:
# Simple runtime checks (no external test files created)
assert isinstance(result, dict), 'Agent result should be a dict'
# The agent returns a structured final report placed in shared['final_report']
assert 'num_conversations' in result, 'Result missing num_conversations key'
print('Basic assertions passed: result appears to be a final report dict')

In [ ]:
# Save the result artifact for inspection
import json
out_path = Path('agent_test_output.json').resolve()
with open(out_path, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)
print('Saved agent output to', out_path)

## Next steps / integration notes

- To run the agent against real GitHub data, remove or replace the monkeypatch cell and set real `collection`, `cache_path` and credentials via environment variables (do not commit tokens).
- If you want me to add optional assertions on `agent.shared` (for example `claim_verification`), tell me which keys you expect and I'll add them.
- If anything in this offline run fails, paste the notebook output and I will debug the failing cell.